In [25]:
import pandas as pd
import nba_api
import altair as alt
import streamlit as st
from itertools import product

In [38]:
# Create the synergy dataframe
synergy_playtypes =['Cut', 'Handoff', 'Isolation', 'Misc', 'OffScreen', 'Postup', 'PRBallHandler', 'PRRollman', 'OffRebound', 'Spotup', 'Transition']
synergy_sides = ['Offense', 'Defense']

def fetch_synergy_data(playtype, side_ball):
    if side_ball == 'Offense':
        side_ball = 'offensive'
    else:
        side_ball = 'defensive'

    synergy_data = nba_api.stats.endpoints.SynergyPlayTypes(
        league_id = '00',
        per_mode_simple = 'Totals',
        season = '2024-25',
        season_type_all_star = 'Regular Season',
        player_or_team_abbreviation = 'T',
        type_grouping_nullable = side_ball,
        play_type_nullable = playtype
    ).get_data_frames()[0]

    return synergy_data

In [39]:
# All combinations: 11 playtypes × 2 sides = 22 rows
synergy_data = pd.concat([
    fetch_synergy_data(playtype, side) for playtype, side in product(synergy_playtypes, synergy_sides)
], ignore_index=True)

synergy_data

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,PLAY_TYPE,TYPE_GROUPING,PERCENTILE,GP,POSS_PCT,PPP,...,TOV_POSS_PCT,SF_POSS_PCT,PLUSONE_POSS_PCT,SCORE_POSS_PCT,EFG_PCT,POSS,PTS,FGM,FGA,FGMX
0,22024,1610612743,DEN,Denver Nuggets,Cut,Offensive,0.931,82,0.100,1.380,...,0.048,0.144,0.039,0.702,0.710,933,1288,557,785,228
1,22024,1610612744,GSW,Golden State Warriors,Cut,Offensive,0.310,82,0.101,1.271,...,0.052,0.158,0.044,0.634,0.638,938,1192,498,780,282
2,22024,1610612739,CLE,Cleveland Cavaliers,Cut,Offensive,1.000,82,0.080,1.427,...,0.050,0.141,0.036,0.717,0.734,743,1060,459,625,166
3,22024,1610612737,ATL,Atlanta Hawks,Cut,Offensive,0.379,82,0.086,1.276,...,0.049,0.127,0.032,0.646,0.658,819,1045,459,698,239
4,22024,1610612761,TOR,Toronto Raptors,Cut,Offensive,0.034,82,0.093,1.166,...,0.065,0.131,0.029,0.591,0.592,875,1020,430,726,296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,22024,1610612763,MEM,Memphis Grizzlies,Transition,Defensive,0.517,82,0.194,1.136,...,0.130,0.121,0.032,0.516,0.625,1863,2116,792,1439,647
656,22024,1610612751,BKN,Brooklyn Nets,Transition,Defensive,0.172,82,0.207,1.156,...,0.127,0.120,0.034,0.519,0.631,1839,2126,787,1428,641
657,22024,1610612764,WAS,Washington Wizards,Transition,Defensive,0.103,82,0.193,1.163,...,0.097,0.112,0.028,0.521,0.614,1829,2127,789,1481,692
658,22024,1610612740,NOP,New Orleans Pelicans,Transition,Defensive,0.000,82,0.198,1.185,...,0.107,0.128,0.027,0.540,0.636,1836,2175,804,1439,635


In [40]:
synergy_data.to_csv('files/synergy_data.csv', index=False)

In [47]:
# Add rank column for each play_type and type_grouping combination
# Rank by GP (Games Played) - you can change this to any column you want to rank by

synergy_offense = synergy_data[synergy_data['TYPE_GROUPING'] == 'Offensive']
synergy_offense['OFF_RANK'] = synergy_offense.groupby(['PLAY_TYPE', 'TYPE_GROUPING'])['PPP'].rank(ascending=False, method='dense')

synergy_defense = synergy_data[synergy_data['TYPE_GROUPING'] == 'Defensive']
synergy_defense['DEF_RANK'] = synergy_defense.groupby(['PLAY_TYPE', 'TYPE_GROUPING'])['PPP'].rank(ascending=True, method='dense')



# Display the data with ranks
synergy_offense[['PLAY_TYPE', 'TYPE_GROUPING', 'TEAM_NAME', 'PPP', 'OFF_RANK']].sort_values(['PLAY_TYPE', 'TYPE_GROUPING', 'OFF_RANK']).reset_index(drop=True)

/var/folders/yz/j96xl3wd5ll5m4p0qwtc4tz00000gn/T/ipykernel_74638/2364116158.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  synergy_offense['OFF_RANK'] = synergy_offense.groupby(['PLAY_TYPE', 'TYPE_GROUPING'])['PPP'].rank(ascending=False, method='dense')
/var/folders/yz/j96xl3wd5ll5m4p0qwtc4tz00000gn/T/ipykernel_74638/2364116158.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  synergy_defense['DEF_RANK'] = synergy_defense.groupby(['PLAY_TYPE', 'TYPE_GROUPING'])['PPP'].rank(ascending=True, method='den

,PLAY_TYPE,TYPE_GROUPING,TEAM_NAME,PPP,OFF_RANK
0,Cut,Offensive,Cleveland Cavaliers,1.427,1.0
1,Cut,Offensive,Chicago Bulls,1.389,2.0
2,Cut,Offensive,Denver Nuggets,1.380,3.0
3,Cut,Offensive,Boston Celtics,1.364,4.0
4,Cut,Offensive,LA Clippers,1.358,5.0
...,...,...,...,...,...
325,Transition,Offensive,Brooklyn Nets,1.081,23.0
326,Transition,Offensive,Washington Wizards,1.072,24.0
327,Transition,Offensive,Toronto Raptors,1.051,25.0
328,Transition,Offensive,Portland Trail Blazers,1.026,26.0


In [42]:
synergy_data.columns

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'PLAY_TYPE',
       'TYPE_GROUPING', 'PERCENTILE', 'GP', 'POSS_PCT', 'PPP', 'FG_PCT',
       'FT_POSS_PCT', 'TOV_POSS_PCT', 'SF_POSS_PCT', 'PLUSONE_POSS_PCT',
       'SCORE_POSS_PCT', 'EFG_PCT', 'POSS', 'PTS', 'FGM', 'FGA', 'FGMX'],
      dtype='object')

In [ ]:
# Add rank column for each play_type and type_grouping combination
# Rank by GP (Games Played) - you can change this to any column you want to rank by
synergy_data['rank'] = synergy_data.groupby(['PLAY_TYPE', 'TYPE_GROUPING'])['GP'].rank(ascending=False, method='dense')

# Display the data with ranks
synergy_data[['PLAY_TYPE', 'TYPE_GROUPING', 'TEAM_NAME', 'GP', 'rank']].sort_values(['PLAY_TYPE', 'TYPE_GROUPING', 'rank'])
